In [ ]:
%load_ext autoreload
%autoreload 2

import edunn as nn
import matplotlib.pyplot as plt
import numpy as np

from edunn import utils

# Parameter initialization

The previously created layers `AddConstant` and `MultiplyConstant` were not defined as _layers with parameters_, they were defined taking a fixed value provided at creation time.

If defined as _layer with parameters_, the parameters can be initialized in a more flexible fashion (e.g.: setting random values) using an `Initializer` object.

# Initializer
An `Initializer` object allows to delegate the responsibility of parameter initialization in a layer. Different initialization strategies can be defined, for example:

- Initialization with a constant value
- Initialization with 0 (special case of the previous one)
- Initialization with random values

# Creation and Initialization

In this guide we'll create a `DummyLayer` layer to understand the usage of the `Initializer`.

The `DummyLayer` layer has a parameters vector `c` that must be initialized somehow. Also this parameter is registered in the layer to allow further access to it.
This layer works for arrays with the same size as the parameters.

Take a look at the implementation of the `__init__` method from the `DummyLayer` layer to see how the parameter `c` is created. You'll observe that an `Initializer` is used to set its initial value.

When creating the layer, it can get an object of the `Initializer` class, which will create and assign the initial value to the parameter `c`. By default, if no `Initializer` is passed, the parameter will be initialized with zeroes with the class `initializers.Zero`.


In [ ]:
from edunn.model import Model, ParameterSet
from edunn import initializers


class DummyLayer(Model):

    def __init__(self, output_size: int, initializer: initializers.Initializer = None, name=None):
        super().__init__(name=name)

        if initializer is None:
            initializer = initializers.Zero()

        c = initializer.create((output_size,))
        self.register_parameter('c', c)

    def forward(self, *x) -> np.ndarray:
        pass

    def backward(self, dE_dy: np.ndarray) -> (np.ndarray, ParameterSet):
        pass



Looking at the implementation of the class `Initializers.Zero` in `edunn/initializers.py` we can see that:
* Inherits from `Initializer`
* Implements the method `initialize(self, p: np.ndarray)` which gets a numpy array for initialization
* Uses `p[:]` to initialize with 0 instead of `p = 0`. There are two main reasons for this:
    * Using `p = 0` would only change the _local variable_ `p` instead of changing the _numpy array_ to which `p` points to
    * When using `p[:]` we're modifying the __contents__ of the parameter array, which belongs to the layer's class(en `DummyLayer` in the example)

Once the class is created we can get the parameter vector `c` form the class `DummyLayer` calling the method `get_parameters()`


In [ ]:
# Create a DummyLayer layer with 2 input/output values
layer = DummyLayer(2, initializer=nn.initializers.Zero())
print(f"Layer name: {layer.name}")
print(f"Layer parameters: {layer.get_parameters()}")
print()

# By default, the initializer is already `Zero`
layer2 = DummyLayer(2)
print(f"Layer name: {layer2.name}")
print(f"Layer parameters: {layer2.get_parameters()}")

# Accessing parameters by name

The `get_parameters()` method returns a dictionary of parameters, because a layer can have more than one parameter. 

Given that we already know the name of the only parameter in this layer, we can access it by its name in string form, `'c'`:

In [ ]:
print(f"Layer parameters: {layer.get_parameters()}")
print(f"Layer parameter 'c': {layer.get_parameters()['c']}")


# Implementation of a Constant Initializer

While sometimes parameters are initialized to `0`, it's common to initialize them with some constant value.

Implement the `Constant` initializer that assigns a constant value or array to the parameter. This allows, for example, initializing `c` with all values of `3` or with a vector of values `[1, 2, 3, 4]`.

Find the `Constant` class in the `edunn/initializers.py` module and implement the `initialize` method.



In [ ]:
# Create a DummyLayer layer with 2 output values (and input values as well)
# All parameters are initialized to 3
value = 3
layer = DummyLayer(2, initializer=nn.initializers.Constant(value))

print(f"Layer Name: {layer.name}")
print(f"Layer Parameter 'c': {layer.get_parameters()['c']}")
utils.check_same(layer.get_parameters()['c'], np.array([3, 3]))
print()

# Create a DummyLayer layer with initial values 1, 2, 3, 4. 
# Note that we are ensuring that the number of values of the Constant initializer match those of the value array

value2 = np.array([1, 2, 3, 4])
layer2 = DummyLayer(4, initializer=nn.initializers.Constant(value2))

print(f"Layer 2 Name: {layer2.name}")
print(f"Layer 2 Parameter 'c': {layer2.get_parameters()['c']}")

utils.check_same(layer2.get_parameters()['c'], value2)

# Implementation of Random Initializers

It's a common practice to initialize the parameters with values sampled from some distribution. In this section we'll work with different random initializers.


## RandomUniform Initializer

Look for the `RandomUniform` class in the `edunn/initializers.py` module and implement the `initialize` method to initialize the parameter with random values sampled form an Uniform distribution.


In [ ]:
# Create two DummyLayer layers with a parameter size of 5

uniform_value_a = 1e-10
dimension = 100
layer1 = DummyLayer(dimension, initializer=nn.initializers.RandomUniform(uniform_value_a))
print(f"Layer1 name: {layer1.name}")
print(f"Layer1 parameters: {layer1.get_parameters()}")

layer2 = DummyLayer(dimension, initializer=nn.initializers.RandomUniform(uniform_value_a))
print(f"Layer2 name: {layer2.name}")
print(f"Layer2 parameters: {layer2.get_parameters()}")

print("(these values should change each time you run this cell)")
print()

c1 = layer1.get_parameters()['c']
c2 = layer2.get_parameters()['c']

# Plot the parameter distribution to see that it has a Uniform distribution shape
plt.hist([c1, c2], bins=dimension // 10)

print("Check that two layers have different initial values for c:")
utils.check_different(c1, c2)


## RandomNormal Initializer

Look for the `RandomUniform` class in the `edunn/initializers.py` module and implement the `initialize` method to initialize the parameter with random values sampled form a Normal distribution with mean `0` and a standard deviation that can be configured upon creation.


In [ ]:
# Create two DummyLayer layers with a parameter size of 5

std = 1e-12
dimension = 100
layer1 = DummyLayer(dimension, initializer=nn.initializers.RandomNormal(std))
print(f"Layer1 name: {layer1.name}")
print(f"Layer1 parameters: {layer1.get_parameters()}")

layer2 = DummyLayer(dimension, initializer=nn.initializers.RandomNormal(std))
print(f"Layer2 name: {layer2.name}")
print(f"Layer2 parameters: {layer2.get_parameters()}")

print("(these values should change each time you run this cell)")
print()

c1 = layer1.get_parameters()['c']
c2 = layer2.get_parameters()['c']

# Tolerance defined as per the 95% confidence interval
# Keep in mind that it's statistically feasible to get a failure scenario.
# You can try running this cell multiple times
tolerance = 1.96 * std / (dimension ** 0.5)

# Plot the parameter distribution to see that it has a Normal distribution shape
plt.hist([c1, c2], bins=dimension // 10)

print("Check that two layers have different initial values for c:")
utils.check_different(c1, c2)
